In [2]:
import pandas as pd
import numpy as np
import boto3

client = boto3.client('s3')

bucket='asdi-hackathon'

file_key1 = 'population-data/popcsv/longitude0.csv'
file_key2 = 'population-data/popcsv/longitudeNeg10.csv'

csv_obj1 = client.get_object(Bucket=bucket, Key=file_key1)
csv_obj2 = client.get_object(Bucket=bucket, Key=file_key1)

obj1 = client.get_object(Bucket=bucket, Key=file_key1)
obj2 = client.get_object(Bucket=bucket, Key=file_key2)

df1 = pd.read_csv(obj1['Body'])
df2 = pd.read_csv(obj2['Body'])

IncompleteReadError: 12321034 read, but total bytes expected is 72161952.

In [4]:
#check data types, float64 or float32?
df1.dtypes

Unnamed: 0      int64
latitude      float64
longitude     float64
population    float64
dtype: object

In [3]:
df1.head()

,Unnamed: 0,latitude,longitude,population
0,1308823,51.560139,0.227083,5.40624
1,1308824,51.560694,0.226250,5.40624
2,1308825,51.560417,0.226250,5.40624
3,1308826,51.560417,0.228472,5.40624
4,1308827,51.559861,0.227361,5.40624


In [6]:
df2.head()

NameError: name 'df2' is not defined

In [7]:
df = df1.append(df2)
df = df.drop('Unnamed: 0', axis = 1)
df

NameError: name 'df2' is not defined

In [15]:
#check for duplicates
print(df.duplicated(subset = ['latitude', 'longitude'], keep=False).sum())

0


In [17]:
#as data high resolution, using knn model to nearest neighbour search takes an eternity
#converting from float64 to float32 both reduces memory and compute time
#there should be minimal loss in precision, besides for population precision loss isn't as important as for likes of latitude and longitude
df['population'] = df['population'].astype(np.float32)

df.dtypes

Unnamed: 0      int64
latitude      float64
longitude     float64
population    float32
dtype: object

In [ ]:
print(max(df['latitude']))
min(df['latitude'])

51.69291666666667


51.26513888888889

In [ ]:
print(max(df['longitude']))
min(df['longitude'])

0.56125


-0.7465277777777778

In [ ]:
nrow_before = len(df)
df_filter = df[(df['latitude'] > 51.239405) & (df['latitude'] < 51.737184)]
df_filter = df_filter[(df_filter['longitude'] > -0.625211) & (df_filter['longitude'] < 0.328289)]
nrow_after = len(df_filter)
print('Number of rows removed:', nrow_before - nrow_after)
df = df_filter

Number of rows removed: 264834


In [ ]:
df

,latitude,longitude,population
0,51.560139,0.227083,5.406240
1,51.560694,0.226250,5.406240
2,51.560417,0.226250,5.406240
3,51.560417,0.228472,5.406240
4,51.559861,0.227361,5.406240
...,...,...,...
1148462,51.554583,-0.208472,33.506246
1148463,51.553750,-0.207639,33.506246
1148464,51.554028,-0.207639,33.506246
1148465,51.554861,-0.208750,33.506246


In [ ]:
#haversine requires latitude and longitude in radians
import math
df['latitude'] = df['latitude'].apply(math.radians)
df['longitude'] = df['longitude'].apply(math.radians)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

#use all of data in train
X_train = df[['latitude', 'longitude']]
y_train = df['population'].ravel()

#use best k value
#lower k value works better for accuracy's sake for high resolution datasets such as pop density
k = 1
model = KNeighborsRegressor(n_neighbors=k, weights = 'distance', algorithm = 'kd_tree', metric = 'haversine', n_jobs = -1)
model.fit(X_train, y_train)

KNeighborsRegressor(algorithm='brute', metric='haversine', n_jobs=-1,
                    n_neighbors=1, weights='distance')

In [ ]:
#pickle final model
import pickle
filename = 'popdensity_model.pkl'
model = model
pickle.dump(model, open(filename, 'wb'))